In [1]:
#Dates for the periods

#Dates for the most recent period
start_date = '2023-07-01'
end_date = '2023-09-30'
#Dates for the period before the most recent period
start_date_last = '2023-04-01'
end_date_last = '2023-06-30'

#Schooling year
year_schooling = ['2022-2023']


### Importing packages

In [2]:
#Importing packages

import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.chrome.options import Options
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from dotenv import load_dotenv
import openpyxl


### Downloading Household children file from Commcare

In [3]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('EMAIL')
password_cc = os.getenv('PASSWORD')

#Defining the driver
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(1000)

#Creating login function
def commcare_login():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/f6ddce2133f8d233d9fbd9341220ed6f/'
    )
    driver.find_element("xpath", '//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element("xpath", '//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR, 'button[type=submit]').click()
       
commcare_login()

#Download the file: "Household child"
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

# Nouvelle forme appel OEV
def appel_oev():

    driver.get(
         'https://www.commcarehq.org/a/caris-test/data/export/custom/new/form/download/d24fc1a47af7508393152bf966a22f99/'
    )

appel_oev()

#Download the database "forme appel OEV"
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

#Positive kids Schooling Information
def positive_kids():
    driver.get(
    "https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/67352311fba5d3a5e285283aa4951676/"
    )
    
positive_kids()

#Download positive database
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()


#-------------------------------------------------------------------------------------------------------------------------

#Gardening Information
def commcare_all_gardens():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/789629a97bddd10b4648d5138d17908e/'
    )

#All_gardens
commcare_all_gardens()

#Download the database "All gardens"
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()


True

In [4]:
# Import schooling positive data
school_positive = pd.read_excel(f"~/Downloads/schooling Enfant positif {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", parse_dates = True)
school_pos = school_positive[(school_positive['schooling_year'].isin(year_schooling)) &
                             (school_positive['eskew_peye'] == 'wi') & (school_positive['closed'] != True)]
school_pos = school_pos[['patient_code', 'fullname']]
school_pos.shape[0]


1076

In [5]:
all_gardens = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/All Gardens {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", 
                            parse_dates = True)
all_gardens['info.last_modified_date'] = pd.to_datetime(all_gardens['info.last_modified_date'], errors = 'coerce')

#Creating new site and commune columns
all_gardens['site'] = all_gardens['caris_site'].str[:8]
all_gardens.loc[(all_gardens.site == 'ZLS/STMC'), 'site'] = 'ZLS/SSPE'
all_gardens.loc[(all_gardens.site == 'GRA/CSMN'), 'site'] = 'JER/CSMN'
all_gardens.loc[(all_gardens.site == 'ART/CSAR'), 'site'] = 'GON/CSAR'
all_gardens.loc[(all_gardens.site == 'GRA/HSPC'), 'site'] = 'JER/HSPC'
all_gardens.loc[(all_gardens.site == 'ZLS/POZS'), 'site'] = 'MON/POZS'
all_gardens.loc[(all_gardens.site == 'MAI/CSDM'), 'site'] = 'HIN/CSMD'

#Upper ST-code
all_gardens.rename(columns = {'patient_code': 'code_ST'}, inplace = True)
all_gardens['code_ST'] = all_gardens['code_ST'].str.upper()

#Create full column code
all_gardens['patient_code'] = all_gardens['site'] + '/' + all_gardens['code_ST']

pos_gardens = all_gardens[(all_gardens['beneficiary_type'] == 'Caris') & 
                          ((all_gardens['info.last_modified_date'] >= start_date_last) & 
                           (all_gardens['info.last_modified_date'] <= end_date)) & 
                          (all_gardens['closed'] != True)][['patient_code']]
pos_gardens.shape[0]


1395

In [6]:
#connecting to HIV Haiti
load_dotenv('dot.env')
user = os.getenv('MYSQL_USER')
password_hiv = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DB')

conn_text = f'mysql+pymysql://{user}:{password_hiv}@{host}/{db}'
engine = create_engine(conn_text)


True

### Connecting to HIV Haiti

In [7]:
sql = f'''

SELECT
patient.id as id_patient,
patient_code,
coalesce(ti.dob,tme.infant_dob,spy.date_of_birth) as dob,
coalesce(ti.gender,tme.infant_gender) as gender,
IF(arv.id_patient IS NOT NULL,
   'yes',
   'no') AS on_arv,
IF(statut.id_patient IS NOT NULL,
   'pos',
   'neg') AS statut,
IF(followup_q1 IS NOT NULL
   OR odk_q1 IS NOT NULL
   OR arv_q1 IS NOT NULL
   OR ques_q1 IS NOT NULL
   OR club_q1 IS NOT NULL
   OR mereenfant_q1 is not null
   OR specimen_q1 is not null
   OR rapid.rapid_q1 is not null
   OR opf_q1.id_patient is not null,
   'yes',
   'no') AS q1,
IF(followup_q1 IS NOT NULL, 'yes', 'no') AS followup_q1,
IF(odk_q1 IS NOT NULL, 'yes', 'no') AS odk_q1,
IF(arv_q1 IS NOT NULL, 'yes', 'no') AS arv_q1,
IF(ques_q1 IS NOT NULL, 'yes', 'no') AS ques_q1,
IF(club_q1 IS NOT NULL, 'yes', 'no') AS club_q1,
if(mereenfant_q1 is not null, 'yes','no') as mereenfant_q1,
if(specimen_q1 is not null, 'yes','no') as specimen_q1,
if(rapid.rapid_q1 is not null, 'yes','no') as rapid_q1,
IF(followup_q2 IS NOT NULL
   OR odk_q2 IS NOT NULL
   OR arv_q2 IS NOT NULL
   OR ques_q2 IS NOT NULL
   OR club_q2 IS NOT NULL
   or mereenfant_q2 is not null
   or specimen_q2 is not null
   OR rapid2.rapid_q2 is not null
   OR opf_q2.id_patient is not null,
   'yes',
   'no') AS q2,
IF(followup_q2 IS NOT NULL, 'yes', 'no') AS followup_q2,
IF(odk_q2 IS NOT NULL, 'yes', 'no') AS odk_q2,
IF(arv_q2 IS NOT NULL, 'yes', 'no') AS arv_q2,
IF(ques_q2 IS NOT NULL, 'yes', 'no') AS ques_q2,
IF(club_q2 IS NOT NULL, 'yes', 'no') AS club_q2,
if(mereenfant_q2 is not null, 'yes','no') as mereenfant_q2,
if(specimen_q2 is not null, 'yes','no') as specimen_q2,
if(rapid2.rapid_q2 is not null, 'yes','no') as rapid_q2,
if(opf_q1.id_patient is not null,'yes','no') as odk_phone_followup_q1,
if(opf_q2.id_patient is not null,'yes','no') as odk_phone_followup_q2,
patient.created_at,
lookup_hospital.name as hospital_name,
office,
IF(hn.id,'yes','no') as is_ugp,
if(ti.is_abandoned=1, 'yes', 'No') as Abandoned,
if(ti.is_dead=1, 'yes', 'No') as Dead

FROM
(SELECT
  *
    FROM
  (SELECT
    id_patient
    FROM
    tracking_followup
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION
    
    ##### ADDING positive_pcr_1 or positive_pcr_2 START
    SELECT
    id_patient
    FROM
    tracking_infant
    WHERE
    (positive_pcr_1 BETWEEN '{start_date_last}' AND '{end_date}'
      OR positive_pcr_2 BETWEEN '{start_date_last}' AND '{end_date}')
    
    ##### ADDING positive_pcr_1 or positive_pcr_2 END
    
    ### ADDING ODK PHONE FOLLOWUP START
    UNION (
      select pt.id as id_patient from tracking_odk_phone_followup topf 
      left join patient pt on topf.patient_code = pt.patient_code
      where (topf.eccm_date between '{start_date_last}' AND '{end_date}')  and topf.name='Enquette Corona Enfant' and topf.eccm_joignable_par_tel!=0
    )
    UNION
    ### ADDING ODK PHONE FOLLOWUP END
    
    (SELECT DISTINCT
      patient.id AS id_patient
      FROM
      caris_db.odk_child_visit
      LEFT JOIN patient ON patient.patient_code = odk_child_visit.patient_code
      WHERE
      date_of_visit BETWEEN '{start_date_last}' AND '{end_date}'
      AND patient.id IS NOT NULL
      AND is_available_at_time_visit = 1
      AND linked_to_id_patient = 0) UNION SELECT
    id_patient
    FROM
    tracking_regime
    WHERE
    (start_date BETWEEN '{start_date_last}' AND '{end_date}' or end_date BETWEEN '{start_date_last}' AND '{end_date}')
    AND category = 'regime_infant_treatment' UNION SELECT
    id_patient
    FROM
    questionnaire_child
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION SELECT
    id_patient
    FROM
    testing_mereenfant
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION SELECT
    id_patient
    FROM
    testing_specimen
    WHERE
    date_blood_taken BETWEEN '{start_date_last}' AND '{end_date}'  UNION SELECT
    id_patient
    FROM
    questionnaire_proftraining18
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION ((SELECT
                                                              id_patient
                                                              FROM
                                                              session
                                                              LEFT JOIN club_session ON club_session.id = id_club_session
                                                              LEFT JOIN club ON club_session.id_club = club.id
                                                              WHERE
                                                              is_present = 1
                                                              AND (date BETWEEN '{start_date}' AND '{end_date}'
                                                                   OR date BETWEEN '{start_date_last}' AND '{end_date_last}')
                                                              AND club_type != 1))) o) bot
LEFT JOIN
(SELECT DISTINCT
  id_patient AS followup_q1
  FROM
  tracking_followup
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}') x ON followup_q1 = bot.id_patient
left join
(select distinct id_patient as specimen_q1 from testing_specimen WHERE
  date_blood_taken BETWEEN '{start_date_last}' AND '{end_date_last}') x2 ON specimen_q1 = bot.id_patient
left join
(select distinct id_patient as specimen_q2 from testing_specimen WHERE
  date_blood_taken BETWEEN '{start_date}' AND '{end_date}') x3 ON specimen_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS followup_q2
  FROM
  tracking_followup
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}') y ON followup_q2 = bot.id_patient
LEFT JOIN
patient ON patient.id = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS mereenfant_q1
  FROM
  testing_mereenfant
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}') me ON mereenfant_q1 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS mereenfant_q2
  FROM
  testing_mereenfant
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}') me2 ON mereenfant_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  patient.id AS odk_q1
  FROM
  caris_db.odk_child_visit
  LEFT JOIN patient ON patient.patient_code = odk_child_visit.patient_code
  WHERE
  date_of_visit BETWEEN '{start_date_last}' AND '{end_date_last}'
  AND patient.id IS NOT NULL
  AND is_available_at_time_visit = 1
  AND linked_to_id_patient = 0) z ON odk_q1 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  patient.id AS odk_q2
  FROM
  caris_db.odk_child_visit
  LEFT JOIN patient ON patient.patient_code = odk_child_visit.patient_code
  WHERE
  date_of_visit BETWEEN '{start_date}' AND '{end_date}'
  AND patient.id IS NOT NULL
  AND is_available_at_time_visit = 1
  AND linked_to_id_patient = 0) zz ON odk_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS arv_q2
  FROM
  tracking_regime
  WHERE
  start_date BETWEEN '{start_date}' AND '{end_date}' or end_date BETWEEN '{start_date}' AND '{end_date}') a2 ON arv_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS arv_q1
  FROM
  tracking_regime
  WHERE
  start_date BETWEEN '{start_date_last}' AND '{end_date_last}' or end_date BETWEEN '{start_date_last}' AND '{end_date_last}') a1 ON arv_q1 = bot.id_patient
LEFT JOIN
(SELECT
  id_patient AS ques_q1
  FROM
  questionnaire_child
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}' UNION SELECT
  id_patient AS ques_q1
  FROM
  questionnaire_proftraining18
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}') q1 ON ques_q1 = bot.id_patient
LEFT JOIN
(SELECT
  id_patient AS ques_q2
  FROM
  questionnaire_child
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}' UNION SELECT
  id_patient AS ques_q2
  FROM
  questionnaire_proftraining18
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}') q2 ON ques_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS club_q1
  FROM
  session
  LEFT JOIN club_session ON club_session.id = id_club_session
  LEFT JOIN club ON club_session.id_club = club.id
  WHERE
  is_present = 1
  AND date BETWEEN '{start_date_last}' AND '{end_date_last}'
  AND club_type != 1) c1 ON club_q1 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS club_q2
  FROM
  session
  LEFT JOIN club_session ON club_session.id = id_club_session
  LEFT JOIN club ON club_session.id_club = club.id
  WHERE
  is_present = 1
  AND date BETWEEN '{start_date}' AND '{end_date}'
  AND club_type != 1) c2 ON club_q2 = bot.id_patient
LEFT JOIN
lookup_hospital ON CONCAT(lookup_hospital.city_code,
                          '/',
                          lookup_hospital.hospital_code) = LEFT(patient_code, 8)
LEFT JOIN
(SELECT DISTINCT
  id_patient
  FROM
  tracking_regime
  WHERE
  category = 'regime_infant_treatment') arv ON arv.id_patient = bot.id_patient
##### ADDING positive_pcr_1 or positive_pcr_2 START
LEFT JOIN (SELECT
           id_patient as rapid_q1
           FROM
           tracking_infant
           WHERE
           (positive_pcr_1 BETWEEN '{start_date_last}' AND '{end_date_last}'
             OR positive_pcr_2 BETWEEN '{start_date_last}' AND '{end_date_last}')) rapid on rapid.rapid_q1=bot.id_patient

##### ADDING positive_pcr_1 or positive_pcr_2 END

##### ADDING positive_pcr_1 or positive_pcr_2 START
LEFT JOIN (SELECT
           id_patient as rapid_q2
           FROM
           tracking_infant
           WHERE
           (positive_pcr_1 BETWEEN '{start_date}' AND '{end_date}'
             OR positive_pcr_2 BETWEEN '{start_date}' AND '{end_date}')) rapid2 on rapid2.rapid_q2=bot.id_patient

##### ADDING positive_pcr_1 or positive_pcr_2 END

### ODK PHONE FOLLOWUP START

LEFT JOIN (
  select case_id,eccm_date, pt3.id as id_patient from tracking_odk_phone_followup topf
  left join patient pt3 on  pt3.patient_code = topf.patient_code
  where topf.eccm_joignable_par_tel!=0 and topf.eccm_date BETWEEN '{start_date_last}' AND '{end_date_last}' group by topf.case_id
) opf_q1 on opf_q1.id_patient= bot.id_patient
LEFT JOIN (
  select case_id,eccm_date, pt4.id as id_patient from tracking_odk_phone_followup topf
  left join caris_db.odk_hivhaiti_backward_compatibility obc on obc.odk_case_id=topf.case_id
  left join patient pt4 on pt4.patient_code = topf.patient_code
  where topf.eccm_joignable_par_tel!=0 and topf.eccm_date BETWEEN '{start_date}' AND '{end_date}' group by topf.case_id
) opf_q2 on opf_q2.id_patient= bot.id_patient
### ODK PHONE FOLLOWUP END
LEFT JOIN
(SELECT
  id_patient
  FROM
  tracking_followup UNION SELECT
  id_patient
  FROM
  tracking_regime
  WHERE
  category = 'regime_infant_treatment' UNION SELECT
  id_patient
  FROM
  club_patient UNION SELECT
  id_patient
  FROM
  questionnaire_child UNION SELECT
  id_patient
  FROM
  questionnaire_proftraining18 UNION SELECT
  id_patient
  FROM
  testing_specimen
  WHERE
  pcr_result = 1 UNION SELECT
  id_patient
  FROM
  testing_result
  WHERE
  result = 1 UNION SELECT
  id_patient
  FROM
  tracking_infant
  WHERE
  positive_pcr_1 != '0000-00-00'
  OR positive_pcr_2 != '0000-00-00') statut ON statut.id_patient = bot.id_patient

left join tracking_infant ti on ti.id_patient=bot.id_patient
left join testing_mereenfant tme on tme.id_patient=bot.id_patient
left join (select max(spx.date_of_birth) as date_of_birth, spx.id_patient from testing_specimen spx group by spx.id_patient) spy on spy.id_patient=bot.id_patient
left join hospital_network hn on (hn.id_hospital=lookup_hospital.id and id_network=7)

WHERE
linked_to_id_patient = 0

'''

#Reading the OVC Helper Mother file
data = pd.read_sql_query(sql, engine, parse_dates = True)
data.shape[0]


3479

In [8]:
#Information sur les sites
info = '''
SELECT concat(city_code,"/",hospital_code)as Site, lh.name as Hospital, ld.name as Departement, lc.name as Commune, ln.name as network FROM caris_db.lookup_hospital lh
left join lookup_commune lc  on lc.id=lh.commune
left join lookup_departement ld on ld.id=lh.departement
left join lookup_network ln on ln.id=lh.network
'''

infos = pd.read_sql_query(info, engine, parse_dates = True)

infos.rename(columns = {'Site': 'site',
                       'Hospital': 'hospital',
                       'Departement': 'Department'}, inplace = True)
infos.columns


Index(['site', 'hospital', 'Department', 'Commune', 'network'], dtype='object')

In [9]:
# data.to_excel('OEV_servis_Q2FY23_02222023.xlsx', index = True)


### Preparing OVC data

In [10]:
#Creating the sites column to merge with the other datasets
data['site'] = data['patient_code'].apply(lambda x: x[0:8])

#converting dob column into datetime type
data['dob'] = pd.to_datetime(data['dob'], errors = 'coerce')

#Adding some required columns
data_merge = data.merge(infos, on = 'site', how = 'left')
data_on_hivhaiti = data_merge

#filtering for patient on HIV Haiti and not belonging to UGP
# data_on_hivhaiti=data_merge[data_merge['Dead']=='No']
# data_on_hivhaiti=data_on_hivhaiti[data_on_hivhaiti['Abandoned']=='No']

#Creating age column
data_on_hivhaiti['dob'].fillna('1901-01-01', inplace = True)
#data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1775-01-01'), 'dob'] = '1901-01-01'


#Transforming dob into datetime type
data_on_hivhaiti['dob'] = pd.to_datetime(data_on_hivhaiti['dob'], errors = 'coerce')

#Creating the age column
def get_age(date):
        return relativedelta(parse(end_date), date).years

data_on_hivhaiti['age'] = data_on_hivhaiti['dob'].map(get_age)

#OVC >= 18
data_sup18 = data_on_hivhaiti[(data_on_hivhaiti['age'] >= 18) & 
                           ((data_on_hivhaiti['patient_code'].isin(school_pos['patient_code'])) | 
                           (data_on_hivhaiti['patient_code'].isin(pos_gardens['patient_code'])))]

#OVC < 18
data_inf18 = data_on_hivhaiti[data_on_hivhaiti['age'] < 18]

#All OVC data
ovc_data = pd.concat([data_inf18, data_sup18])

#ovc_data.head(5)
ovc_data.shape


(2953, 38)

In [11]:
#new calls file
appel = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/Caris Health Agent - Enfant - APPELS OEV (created 2022-03-28) {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
appel.rename(columns = {'form.appels_oev.date_appel': 'date_appel',
                       'form.appels_oev.parenttuteur_trouve': 'found',
                       'form.appels_oev.patient_code': 'patient_code'}, inplace = True)
appel['date_appel'] = pd.to_datetime(appel['date_appel'], errors = 'coerce')
appel['found'] = pd.to_numeric(appel['found'], errors = 'coerce')

appel_qi = appel[(appel['date_appel'] >= start_date_last) & (appel['date_appel'] <= end_date_last) &
                 (appel['found'] == 1)][['patient_code', 'date_appel', 'found']].drop_duplicates('patient_code')
appel_qj = appel[(appel['date_appel'] >= start_date) & (appel['date_appel'] <= end_date) & 
                 (appel['found']) == 1][['patient_code', 'date_appel', 'found']].drop_duplicates('patient_code')

#====================================================================================================
#create table alike services table
all_appel_served_q1q2 = appel_qi.merge(appel_qj[['patient_code', 'date_appel']], on = 'patient_code', how = 'left')

#====================================================================================================

print(f"Nombre d'appels Qi: {appel_qi.shape[0]}")
print(f"Nombre d'appels Qj: {appel_qj.shape[0]}")
print(f"Servis en Q1 et Q2: {all_appel_served_q1q2[all_appel_served_q1q2['date_appel_y'].notnull()].shape[0]}")


Nombre d'appels Qi: 1092
Nombre d'appels Qj: 916
Servis en Q1 et Q2: 630


In [12]:
# create new columns to receive calls information
ovc_data['odk_call_q1'] = "no"
ovc_data["odk_call_q2"] = "no"

# make changes in services according to calls received by beneficiaries
for code in appel_qi['patient_code']:
    ovc_data.loc[(ovc_data['patient_code'] == code), 'odk_call_q1'] = 'yes'

for code in appel_qj['patient_code']:
    ovc_data.loc[(ovc_data['patient_code'] == code), 'odk_call_q2'] = 'yes'

ovc_data.loc[(ovc_data['odk_call_q1'] == 'yes'), 'q1'] = 'yes'
ovc_data.loc[(ovc_data['odk_call_q2'] == 'yes'), 'q2'] = 'yes'


In [13]:
#Served children for the 1st semester

#-------------------------------------------------------------------------------------------------------------------------

#served children in Q1 and Q2
q1q2 = ovc_data[(ovc_data.q1 == 'yes') & (ovc_data.q2 == 'yes') & (ovc_data.network!='UGP')][['patient_code', 'dob','gender','club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#served children created while Q2
q2 = ovc_data[(ovc_data.created_at >= start_date) & (ovc_data.created_at <= end_date) & (ovc_data.network!='UGP')][['patient_code',  'dob','gender','club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#ugp-served children (specimen or mereenfant) in q1 and q2
ugp_data=ovc_data[ovc_data['network'] == 'UGP']
# ugp_data = ugp_data[ugp_data['is_ugp'] == 'yes']
ugp_q1q2=ugp_data[((ugp_data['mereenfant_q1']=='yes') | (ugp_data['specimen_q1']=='yes')) & ((ugp_data['mereenfant_q2']=='yes') |
                    (ugp_data['specimen_q2']=='yes'))][['patient_code', 'dob','gender', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#ugp-served children (specimen or mereenfant) in q2
ugp_q2=ugp_data[(ovc_data.created_at >= start_date) & (ovc_data.created_at <= end_date)]
ugp_q2=ugp_q2[(ugp_q2['mereenfant_q2']=='yes') | (ugp_q2['specimen_q2']=='yes')][['patient_code',  'dob','gender', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]


#Concatenation of the two previous datasets
OVC = pd.concat([q1q2, q2,ugp_q1q2,ugp_q2]).drop_duplicates('patient_code').reset_index(drop = True)
print(f"\nNumber of children (OVC-)served for the semester: {OVC.shape[0]}.")

#=======================================================================================================================

#Function to get the age range
def age_range(age):
    if age < 1:
        return '< 1'
    elif age in range(1, 5):
        return '1 - 4'
    elif age in range(5, 10):
        return '5 - 9'
    elif age in range(10, 15):
        return '10 - 14'
    elif age in range(15, 18):
        return '15 - 17'
    elif age in range(18, 21):
        return '18 - 20'
    else:
        return '21+'

OVC['age_range'] = OVC['age'].map(age_range)
OVC1 = OVC[OVC['age_range'] != '21+']
#==================================================================================

OVC1.head()



Number of children (OVC-)served for the semester: 2351.


,patient_code,dob,gender,club_q1,club_q2,on_arv,Commune,age,age_range
0,PDP/CMEB/ST01108,2006-07-24,1.0,yes,yes,yes,Port-de-Paix,17,15 - 17
1,PDP/CMEB/ST03288,2011-09-26,2.0,yes,yes,yes,Port-de-Paix,12,10 - 14
2,PAP/HSFS/ST01282,2008-02-08,1.0,yes,yes,yes,Port-au-Prince,15,15 - 17
3,PAP/HSFS/ST00954,2015-11-16,1.0,yes,yes,yes,Port-au-Prince,7,5 - 9
4,PAP/OBCG/ST00249,2017-03-13,1.0,yes,yes,yes,Carrefour,6,5 - 9


In [17]:
ovc_data.to_excel("All_Served_Q3_Q4_10172023.xlsx", index = False)


In [18]:
sql2 = '''

select Patient_code,Code_mere,if(Code_mere in (select patient_code from patient),'yes','no') as Liaison_mere,left(patient_code,8) as Site_or_lab_code, hospital,Office, if(Network is not null,Network,'Autres') as Network, date_of_birth, gender, start_date as arv_start, if(Office = 'CAP', 'HUJ', lab) as lab, date_blood_taken, date_caris_received_sample, date_lab_received_sample, Result,
if(timestampdiff(day,date_of_birth,date_blood_taken)<=61,if(timestampdiff(day,date_of_birth,date_blood_taken)>=0,'0_2',''),if(timestampdiff(day,date_of_birth,date_blood_taken)<=365,if(timestampdiff(day,date_of_birth,date_blood_taken)>61,'2_12',''),'')) as tranche_age
,if(arv is not null,'yes','') as on_arv, timestampdiff(day,date_lab_received_sample,now()) as total_jours_Labo, timestampdiff(day,date_blood_taken,pcr_result_date) as from_specimen_to_result, timestampdiff(day,date_blood_taken,date_caris_received_sample) as from_specimen_to_caris,
timestampdiff(day,date_caris_received_sample,date_lab_received_sample) as from_caris_to_lab, timestampdiff(day,date_lab_received_sample,pcr_result_date) as from_lab_to_result,

if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab")) as days_in_lab,
case 
	when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))>15 then 'greater than 15'
    when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))<=15 then 'less or equal to 15'
    else'not in lab' end as Nbres_jours_Labo
   , 
   #timestampdiff(day,date_caris_received_sample,now()) as Total_jours_Caris,
   
if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) as days_between_caris_lab,
case 
	when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00') ,timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) >5 then 'greater than 5'
    when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) <=5 then 'less or equal to 5'
 end as Nbres_jours_Caris,

if(pcr_result=1,if(patient_info is not null,if(is_abandoned=0 and is_dead=0,'no','yes'),''),'') as pos_dead_abandon
,death_date
,abandoned_date
,pcr_result_date,
case 
    when month(date_blood_taken) between 10 and 12 then 'Q1'
    when month(date_blood_taken) between 1 and 3 then 'Q2'
    when month(date_blood_taken) between 4 and 6 then 'Q3'
    else 'Q4'
 End as Quarter,  -- month(date_blood_taken)  as month_blood_taken,
 #year(date_blood_taken) as year_test,
if( month(date_blood_taken) between 10 and 12 ,year(date_blood_taken)+1,year(date_blood_taken) ) as Year
from(
select patient_code,tme.id_patient as id_in_mere ,concat(tme.mother_city_code,'/',tme.mother_hospital_code,'/',tme.mother_code) as Code_mere, date_of_birth, tracking_infant.id_patient as patient_info, date_blood_taken, pcr_result,arv.id_patient as arv, is_dead, is_abandoned, death_date, abandoned_date, pcr_result_date, date_caris_received_sample, date_lab_received_sample,
lookup_testing_specimen_result.name as _result, if(lookup_testing_specimen_result.name="", 'En cours',lookup_testing_specimen_result.name) as Result,lookup_hospital.name as hospital,lookup_network.name as Network,lookup_hospital.office as Office, lookup_testing_gender.name as gender, tracking_regime.start_date, lookup_lab.name as lab from(
    select * from testing_specimen t1 where timestampdiff(day,date_of_birth,date_blood_taken)<=365 and date_blood_taken between '2014-10-01' and '2030-12-31'
    and date_blood_taken=(select min(date_blood_taken) from testing_specimen t2 where t1.id_patient=t2.id_patient)
)test_1
left join patient pa on pa.id=test_1.id_patient
left join testing_mereenfant tme on pa.id= tme.id_patient
left join
(select * from(
    select id_patient from tracking_followup where on_arv=1
    union
    select id_patient from tracking_regime where category='regime_infant_treatment'
    )o
)arv on arv.id_patient=test_1.id_patient
left join tracking_infant on tracking_infant.id_patient=test_1.id_patient
left join lookup_testing_specimen_result on pcr_result=lookup_testing_specimen_result.id
left join lookup_hospital on concat(lookup_hospital.city_code,'/',lookup_hospital.hospital_code)=left(patient_code,8)
left join tracking_regime on tracking_regime.id_patient=test_1.id_patient
left join lookup_testing_gender on lookup_testing_gender.id=gender
left join lookup_lab on lookup_hospital.id_lab=lookup_lab.id
left join lookup_network on lookup_hospital.network=lookup_network.id
where timestampdiff(day,test_1.date_of_birth,test_1.date_blood_taken)<=365 and test_1.date_blood_taken between '2014-10-01' and '2030-12-31' and linked_to_id_patient=0
group by test_1.id_patient) final
'''

eid = pd.read_sql_query(sql2, engine, parse_dates = True)


In [40]:
eid['date_blood_taken'] = pd.to_datetime(eid['date_blood_taken'], errors = 'coerce')

#EID for Q1 and Q2 FY23
eid_fy23 = eid[(eid['date_blood_taken'] >= parse(start_date_last)) & (eid['date_blood_taken'] <= parse(end_date))]

#Negative EID
eid_neg = eid_fy23[eid_fy23['Result'] == 'ADN VIH-1 Non-Detècté']

#Positive EID
eid_pos = eid_fy23[eid_fy23['Result'] == 'ADN VIH-1 Detècté']

#Positive on ARV
eid_arv = eid_pos[eid_pos['on_arv'] == 'yes']

#Positive not on ARV
eid_no_arv = eid_pos[eid_pos['on_arv'] != 'yes']

#Status unknown
eid_no_stat = eid_fy23[~eid_fy23['Result'].isin(['ADN VIH-1 Detècté', 'ADN VIH-1 Non-Detècté'])]

# eid_fy23.shape
# eid_neg.shape
# eid_pos.shape
# eid_arv.shape
# eid_no_arv.shape
# eid_no_stat.shape


In [41]:
eid_fy23.to_excel("EID_data.xlsx", index = False)


In [42]:
OVC1['eid'] = 'no'
for code in eid_fy23['Patient_code']:
    OVC1.loc[(OVC1['patient_code'] == code), 'eid'] = 'yes'

OVC1['status'] = 'positive'
for code in eid_neg['Patient_code']:
    OVC1.loc[(OVC1['patient_code'] == code), 'status'] = 'negative'
    
for code in eid_no_stat['Patient_code']:
    OVC1.loc[(OVC1['patient_code'] == code), 'status'] = 'No_HIV_reported_Status_Unknown'

OVC1.loc[(OVC1['patient_code'] == 'ART/HASD/ST00383'), 'status'] = 'No_HIV_reported_Status_Unknown'

for code in eid_no_arv['Patient_code']:
    OVC1.loc[(OVC1['patient_code'] == code), 'on_arv'] = 'no'


In [43]:
OVC1.shape[0]
OVC1.head(2)


2303

,patient_code,dob,gender,club_q1,club_q2,on_arv,Commune,age,age_range,eid,status
0,PDP/CMEB/ST01108,2006-07-24,Male,yes,yes,yes,Port-de-Paix,17,15 - 17,no,positive
1,PDP/CMEB/ST03288,2011-09-26,Female,yes,yes,yes,Port-de-Paix,12,10 - 14,no,positive


In [44]:
OVC1.to_excel('OVC_Served_FY23Q4_10172023.xlsx', index = False)


In [45]:
OVC1.loc[(OVC1.gender == 1), 'gender'] = 'Male'
OVC1.loc[(OVC1.gender == 2), 'gender'] = 'Female'
OVC1.loc[(OVC1.gender == 0), 'gender'] = 'Female'
OVC1['gender'].fillna('Female', inplace = True)

oev = OVC1.pivot_table(index = 'Commune', columns = ['gender', 'age_range'], values = 'patient_code', 
                              aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18 - 20'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18 - 20')]
oev = oev.reindex(columns = columns)
# index_ff[('Female', '18 - 20')] = 0
# index_ff[('Male', '18 - 20')] = 0
oev.reset_index(inplace = True)
oev


gender             Commune Female                                     Male  \
age_range                     < 1 1 - 4 5 - 9 10 - 14 15 - 17 18 - 20  < 1   
0               Anse-Rouge      0     0     0       0       0       0    0   
1          Anse-d'Hainault      1     2     1       6       1       0    2   
2            Anse-à-Foleur      0     0     3       0       0       0    0   
3            Anse-à-Galets      7     2     3      10       4       1    5   
4              Anse-à-Veau      0     1     0       1       1       0    0   
..                     ...    ...   ...   ...     ...     ...     ...  ...   
62                 Tabarre     14     7    16      56      34      17    4   
63           Terrier-Rouge      1     3     2       2       6       2    1   
64            Trou-du-Nord      5     6     4       7       3       1    6   
65               Verrettes      0     4     0       0       0       0    0   
66                   Total    130   131   274     396     202      76  135   

gender                                         
age_range 1 - 4 5 - 9 10 - 14 15 - 17 18 - 20  
0             0     0       0       1       0  
1             2     0       4       3       0  
2             0     1       0       0       0  
3             2     2       4       4       0  
4             2     0       1       0       0  
..          ...   ...     ...     ...     ...  
62           13    19      47      37      14  
63            0     5       1       0       1  
64            1     7       6       5       1  
65            2     0       0       0       0  
66          140   246     335     179      59  

[67 rows x 13 columns]

# HIVSTAT Direct beneficiaries

In [46]:
#Negative children
ovc_negative_children = OVC1[(OVC1.status == 'negative') & (~OVC1.age_range.isin(['18 - 20', '21+']))]
negative = ovc_negative_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['status', 'gender','age_range'], aggfunc = len, fill_value = 0)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative = negative.reindex(columns = columns)
negative.columns = ['_'.join(col) for col in negative.columns.values]
negative.reset_index(inplace = True)

negative.head()


,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse-d'Hainault,1,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN
1,Anse-à-Galets,7,NaN,NaN,NaN,NaN,5,1,NaN,NaN,NaN
2,Aquin,6,NaN,NaN,NaN,NaN,8,0,NaN,NaN,NaN
3,Arcahaie,1,NaN,NaN,NaN,NaN,5,0,NaN,NaN,NaN
4,Bassin-Bleu,0,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN


In [49]:
# negative.sum()


In [48]:
#positive children
ovc_positive_children = OVC1[(OVC1.status == 'positive') & (~OVC1.age_range.isin(['18 - 20', '21+']))]
positive = ovc_positive_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['status', 'gender','age_range'], aggfunc = len, fill_value = 0)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive = positive.reindex(columns = columns)
positive.columns = ['_'.join(col) for col in positive.columns.values]
positive.reset_index(inplace = True)

positive.head()


,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17
0,Anse-Rouge,0,0,0,0,0,0,0,0,0,1
1,Anse-d'Hainault,0,2,1,6,1,0,2,0,4,3
2,Anse-à-Foleur,0,0,3,0,0,0,0,1,0,0
3,Anse-à-Galets,0,2,3,10,4,0,1,2,4,4
4,Anse-à-Veau,0,1,0,1,1,0,2,0,1,0


In [51]:
# positive.sum()


In [52]:
# Children on arv
ovc_on_arv = ovc_positive_children[ovc_positive_children['on_arv'] == 'yes']
on_arv = ovc_on_arv.pivot_table(values='patient_code', index = 'Commune', columns = ['on_arv', 'gender', 'age_range'], 
                               aggfunc = len, fill_value = 0)
on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
on_arv = on_arv.reindex(columns = columns)
on_arv.columns = ['_'.join(col) for col in on_arv.columns.values]
on_arv.fillna(0, inplace = True)
on_arv.reset_index(inplace = True)
on_arv.head()


,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17
0,Anse-Rouge,0,0,0,0,0,0,0,0,0,1
1,Anse-d'Hainault,0,2,1,6,1,0,2,0,4,3
2,Anse-à-Foleur,0,0,3,0,0,0,0,1,0,0
3,Anse-à-Galets,0,2,3,10,4,0,1,2,4,4
4,Anse-à-Veau,0,1,0,1,1,0,2,0,1,0


In [54]:
# on_arv.sum()


In [55]:
# Children on arv
ovc_not_on_arv = ovc_positive_children[ovc_positive_children['on_arv'] == 'no']
not_on_arv = ovc_not_on_arv.pivot_table(values='patient_code', index = 'Commune', columns = ['on_arv', 'gender', 'age_range'], 
                               aggfunc = len, fill_value = 0)
not_on_arv.rename(columns = {'no': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
not_on_arv = not_on_arv.reindex(columns = columns)
not_on_arv.columns = ['_'.join(col) for col in not_on_arv.columns.values]
not_on_arv.fillna(0, inplace = True)
not_on_arv.reset_index(inplace = True)
not_on_arv.head()


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17
0,Cap-Haïtien,0.0,0.0,0.0,0,0.0,1,0.0,0.0,0,0.0
1,Delmas,0.0,0.0,0.0,0,0.0,0,0.0,0.0,1,0.0
2,Marmelade,0.0,0.0,0.0,1,0.0,0,0.0,0.0,0,0.0
3,Terrier-Rouge,0.0,0.0,0.0,0,0.0,1,0.0,0.0,0,0.0


In [56]:
ovc_unknown_status = OVC1[(OVC1['status'] == 'No_HIV_reported_Status_Unknown') & 
                          (~OVC1['age_range'].isin(['18 - 20', '21+']))]

unknown_status = ovc_unknown_status.pivot_table(values = 'patient_code', index = 'Commune', 
                                                columns = ['status', 'gender', 'age_range'], aggfunc = len, fill_value = 0)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
unknown_status = unknown_status.reindex(columns = columns)
unknown_status.columns = ['_'.join(col) for col in unknown_status.columns.values]
unknown_status.fillna(0, inplace = True)
unknown_status.reset_index(inplace = True)
unknown_status.head()


,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse-d'Hainault,0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0
1,Aquin,0,0.0,0.0,0.0,0.0,2,0,0.0,0.0,0.0
2,Arcahaie,4,0.0,0.0,0.0,0.0,3,0,0.0,0.0,0.0
3,Bassin-Bleu,1,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
4,Carrefour,5,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0


In [58]:
# unknown_status.sum()


In [59]:
ovc_hivstat = positive.merge(on_arv, on = 'Commune', how = 'outer').merge(
    not_on_arv, on = 'Commune', how = 'outer').merge(negative, on = 'Commune', how = 'outer').merge(
unknown_status, on = 'Commune', how = 'outer')

ovc_hivstat.head()


,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse-Rouge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anse-d'Hainault,0.0,2.0,1.0,6.0,1.0,0.0,2.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Anse-à-Foleur,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Anse-à-Galets,0.0,2.0,3.0,10.0,4.0,0.0,1.0,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anse-à-Veau,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# ovc_hivstat.to_excel('OVC_HIVSTAT_OEV_FY23Q4_10132023.xlsx', index = False, na_rep=0)


# Direct beneficiaries - Siblings

In [60]:
# List of direct beneficiaries
#List of children in club for the semester
club_child = OVC1[(OVC1.club_q1 == 'yes') | (OVC1.club_q2 == 'yes')][['patient_code','dob','gender','Commune']]
club_child


,patient_code,dob,gender,Commune
0,PDP/CMEB/ST01108,2006-07-24,Male,Port-de-Paix
1,PDP/CMEB/ST03288,2011-09-26,Female,Port-de-Paix
2,PAP/HSFS/ST01282,2008-02-08,Male,Port-au-Prince
3,PAP/HSFS/ST00954,2015-11-16,Male,Port-au-Prince
4,PAP/OBCG/ST00249,2017-03-13,Male,Carrefour
...,...,...,...,...
2076,PAP/GCHO/ST08382,2016-10-15,Female,Delmas
2077,MON/POZS/ST00934,2008-07-16,Female,Saint-Marc
2079,DMA/HCDM/ST01073,2006-11-06,Male,Anse-d'Hainault
2088,MON/POZS/ST01040,2013-08-24,Male,Saint-Marc


In [61]:
club_child.to_excel("Child_in_club_Q4FY23_10172023.xlsx", index = False)


In [63]:
#Importing household child file
hh_child = pd.read_excel(f"~/Downloads/household_child  {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases', parse_dates = True)

hh_child.full_code_patient_menage = hh_child.full_code_patient_menage.str.upper()

#Importing All patient table from HIV Haiti to know whether the children code is on HIV Haiti or not
sql2 = 'select * from patient'
all_patient = pd.read_sql_query(sql2, engine, parse_dates = True)
all_patient.shape[0]


180005

In [64]:
#Regime ARV all patient
sql3 = '''
SELECT 
       p.patient_code, id_patient,
           MIN(IF(start_date != '0000-00-00', start_date, NULL)) AS start_date,
           GROUP_CONCAT(la.acronym
               SEPARATOR '-') AS regime,
           GROUP_CONCAT(tr.start_date
               SEPARATOR ' | ') AS all_start_date,
           GROUP_CONCAT(tr.end_date
               SEPARATOR ' | ') AS all_end_date
   FROM
       tracking_regime tr
	LEFT JOIN patient p ON p.id = tr.id_patient
   LEFT JOIN lookup_arv la ON la.id = tr.id_arv
   WHERE
    category = 'regime_infant_treatment' and tr.end_date='0000-00-00'
   GROUP BY id_patient
'''

regime = pd.read_sql_query(sql3, engine, parse_dates = True)
regime.rename(columns = {'patient_code': 'full_code_patient_menage',
                        'start_date': 'arv_start_date'}, 
              inplace = True)

regime = pd.read_sql_query(sql3, engine, parse_dates = True)
regime.rename(columns = {'patient_code': 'full_code_patient_menage',
                        'start_date': 'arv_start_date'}, 
               inplace = True)


In [65]:

#----------------------------------------------------------------------------------------------------------------------
#Function to know whether a patient code is on HIV Haiti or not
def on_hivhaiti(code):
    if code.upper() in all_patient.patient_code.tolist():
        return 'yes'
    else:
        return 'no'
    
#----------------------------------------------------------------------------------------------------------------------

#
hh_child.rename(columns = {'main_infant_code':'patient_code'}, 
              inplace = True)

#data for only child in club
hhm_club = hh_child[hh_child['patient_code'].str.lower().isin(club_child['patient_code'].str.lower())]
hhm_club['patient_code'] = hhm_club['patient_code'].str.upper()

#preparing age columns
hhm_club.loc[(hhm_club.age_in_year == '---'), 'age_in_year'] = ''
hhm_club['age_in_year'] = pd.to_numeric(hhm_club['age_in_year'])
hhm_club['age_range'] = hhm_club['age_in_year'].map(age_range)

#column specifying the code is on HIV Haiti
hhm_club['on_HIVHAITI'] = hhm_club['full_code_patient_menage'].map(on_hivhaiti)

#Replacing code by true gender values
hhm_club.loc[(hhm_club.gender == 1), 'gender'] = 'Male'
hhm_club.loc[(hhm_club.gender == 2), 'gender'] = 'Female'


In [66]:
#Adding the communes column
hhm_club = hhm_club.merge(OVC1[['patient_code', 'Commune']], on = 'patient_code', how = 'left')
hhm_club.rename(columns = {'full_code_patient_menage': 'patient_code2'}, inplace = True)
OVC1.rename(columns = {'patient_code': 'patient_code2'}, inplace = True)
hhm_club = hhm_club.merge(OVC1[['patient_code2', 'on_arv']], on = 'patient_code2', how = 'left')
hhm_club['on_arv'].fillna('---', inplace = True)
hhm_club.head(2)
hhm_club.shape


,number,caseid,cty,dob,first_name,gender,hiv_test,hiv_test_date,hiv_test_result,hospital_code,...,code_incorrect_raison,confirmation_code_by_agent,consent,has_schooling_form_2021,school_name,Full_code_patient_menage1,age_range,on_HIVHAITI,Commune,on_arv
0,0,5cdc6db5-3642-4cd2-aa2f-f4c2acf6a5b0,---,1970-08-09,wisly,Male,---,---,---,---,...,---,---,---,---,---,---,21+,no,Dessalines,---
1,71,6933df55-0b9b-4756-8930-e5060dc57349,---,2013-06-03,Manieula,Female,1,2020-03-17,negatif,---,...,---,---,---,---,---,---,5 - 9,no,Jérémie,---


(6229, 46)

In [67]:
hhm_club.to_excel('Comptage_OEV_FY23Q4_10172023.xlsx', index = False)


In [68]:
#creating the siblings dataset and the dataset for the other kids
hhm_ff = hhm_club[hhm_club.infant_relationship == str(3)]
hhm_other = hhm_club[hhm_club.infant_relationship != str(3)]
hhm_other.loc[(hhm_other['name'] == 'Metellus Rose Derlove'), 'gender'] = 'Female'


#Size
hhm_ff_stat = hhm_club[(hhm_club.infant_relationship == str(3)) & (~hhm_club.age_range.isin(['18 - 20', '21+']))]
hhm_other_stat = hhm_club[(hhm_club.infant_relationship != str(3)) & (~hhm_club.age_range.isin(['18 - 20', '21+']))]
hhm_ff_stat.shape

hhm_other_stat.loc[(hhm_other_stat['name'] == 'Metellus Rose Derlove'), 'gender'] = 'Female'
hhm_other_stat.shape


(2230, 46)

(923, 46)

In [69]:
hhm_ff_stat.to_excel('Siblings_FY23Q4_10172023.xlsx', index = False)
hhm_other_stat.to_excel('Other_kids_FY23Q3_10172023.xlsx', index = False)


# DATIM Direct Siblings

In [70]:
#Creating Datim Table for siblings' kids
index_ff = hhm_ff.pivot_table(index = 'Commune', columns = ['gender', 'age_range'], values = 'patient_code', 
                              aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18 - 20'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18 - 20')]
index_ff = index_ff.reindex(columns = columns)
index_ff[('Female', '18 - 20')] = 0
index_ff[('Male', '18 - 20')] = 0
index_ff.reset_index(inplace = True)
index_ff


gender                  Commune Female                                      \
age_range                          < 1 1 - 4 5 - 9 10 - 14 15 - 17 18 - 20   
0               Anse-d'Hainault      0     0     1       3       2       0   
1                 Anse-à-Foleur      0     0     0       1       0       0   
2                 Anse-à-Galets      0     2     2       5       3       0   
3                         Aquin      0     1     1       5       4       0   
4                      Arcahaie      1     4     1       4       8       0   
5                        Bainet      1     1     0       1       1       0   
6                   Cap-Haïtien      1     3     2       3       1       0   
7                     Capotille      0     0     0       0       0       0   
8                       Caracol      0     0     0       0       1       0   
9                        Carice      0     0     1       0       0       0   
10                    Carrefour      0    10    13      22       6       0   
11                    Cavaillon      1     2     3       1       0       0   
12           Croix-des-Bouquets      0     6    40      54      10       0   
13                       Delmas      2    19    66      81      29       0   
14                   Dessalines      0     2     5       3       2       0   
15                       Ennery      0     0     2       3       0       0   
16                     Gonaïves      1     5    10       7       4       0   
17                   Gros-Morne      1     2     9       8       3       0   
18                   Jean-Rabel      0     0     1       0       0       0   
19                      Jérémie      0     2     1       1       0       0   
20                     Kenscoff      1     1     5       1       0       0   
21                      L'Asile      0     1     1       3       0       0   
22                    La Tortue      0     3     8       4       6       0   
23                    Les Cayes      1     0     0       1       1       0   
24                        Limbé      0     1    10       4       4       0   
25                     Limonade      0     0     0       0       0       0   
26                      Léogâne      0     2     0       2       1       0   
27                    Marmelade      0     2     0       4       0       0   
28                     Maïssade      0     0     1       0       0       0   
29                        Milot      0     4     4      11      12       0   
30                    Miragoâne      0     7     7      11       5       0   
31                Mombin-Crochu      0     9     3       8       3       0   
32                Mont-Organisé      0     3     9       8       2       0   
33                  Ouanaminthe      1     5     2       5       4       0   
34         Petit-Trou-de-Nippes      0     0     0       1       0       0   
35                       Pignon      0     2     7       6       3       0   
36                       Pilate      2     3     9      17       5       0   
37               Plaine-du-Nord      0     0     0       5       2       0   
38                    Plaisance      0     0     1       4       0       0   
39                  Port-Margot      0     1     1       1       4       0   
40               Port-au-Prince      1     7    39      44      14       0   
41                 Port-de-Paix      7     8    14      27      15       0   
42          Saint-Louis-du-Nord      0     0     2       0       0       0   
43                   Saint-Marc      0     0     2       6       5       0   
44                Saint-Raphaël      0     2     2       1       1       0   
45                      Tabarre      3    13    33      55      44       0   
46                Terrier-Rouge      3     1     6       4       2       0   
47                 Trou-du-Nord      0     4     7       6       0       0   
48                        Total     27   138   331     441     207       0   

gender    Male       

## OVC_HIVSTAT _ OEV Siblings

In [71]:
#Negative children
# # hhm_negative_children = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'negatif') & (hhm_ff_stat.age_in_year <= 17)]
# hhm_negative_children = hhm_ff_stat_stat[hhm_ff_stat_stat.hiv_test_result == 'negatif']
# negative_children = hhm_negative_children.pivot_table(index = 'Commune', values = 'patient_code', 
#                                                       columns = 'hiv_test_result', aggfunc = len, fill_value = 0)
# negative_children.rename(columns = {'negatif': 'negative'}, inplace = True)
# negative_children.reset_index(inplace = True)
# negative_children.head()



#==========================================================================================================================

hhm_negative_children = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'negatif') & (hhm_ff_stat.age_in_year <= 17)]
negative_children = hhm_negative_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
negative_children.rename(columns = {'negatif': 'negative'}, inplace = True)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative_children = negative_children.reindex(columns = columns)
negative_children.columns = ['_'.join(col) for col in negative_children.columns.values]
negative_children.reset_index(inplace = True)

negative_children.head()


,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse-d'Hainault,0,0,1,2,1,0,2,2,0,0
1,Anse-à-Foleur,0,0,0,0,0,0,0,1,1,0
2,Anse-à-Galets,0,1,2,5,2,0,0,6,5,4
3,Aquin,0,1,1,4,4,0,2,2,5,1
4,Arcahaie,1,4,1,4,5,0,8,12,7,0


In [74]:
hhm_negative_children.shape[0]
# negative_children.sum()


1891

In [41]:
#hhm_positive_children.on_arv.unique()


In [75]:
#Positive children
def on_arv(date):
    if pd.notna(date):
        return 'yes'
    else:
        return 'no'


#hhm_positive_children = hhm_ff[(hhm_ff.hiv_test_result == 'positif') & (hhm_ff.age_in_year <= 17) & (hhm_ff.on_HIVHAITI == 'yes')].drop_duplicates('patient_code2')
hhm_positive_children = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'positif') & (hhm_ff.on_HIVHAITI == 'yes')].drop_duplicates('patient_code2')
positive_children = hhm_positive_children.pivot_table(values = 'patient_code', index = 'Commune', columns = ['hiv_test_result', 'gender','age_range'],
                                                     aggfunc = len, fill_value = 0)
positive_children.rename(columns = {'positif': 'positive'}, inplace = True)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive_children = positive_children.reindex(columns = columns)
positive_children.fillna(0, inplace = True)
positive_children.columns = ['_'.join(col) for col in positive_children.columns.values]


positive_children.reset_index(inplace = True)
positive_children.head()


,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17
0,Delmas,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0
1,Gros-Morne,0.0,0.0,0,0.0,1,0.0,0.0,0.0,0.0,0.0


In [76]:
hhm_positive_children[['patient_code2','patient_code', 'on_arv']]


,patient_code2,patient_code,on_arv
159,GRO/HAMG/00447,GRO/HAMG/ST00938,---
4045,PAP/CMSP/ST01602,PAP/CMSP/ST01563,yes


In [77]:
# hhm_positive_children.to_excel('positive_siblings.xlsx')


In [78]:
#Dataset children on arv
hhm_children_on_arv = hhm_positive_children[hhm_positive_children.on_arv == 'yes']

children_on_arv = hhm_children_on_arv.pivot_table(values = 'patient_code', index = 'Commune', 
                                                  columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)

children_on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
children_on_arv = children_on_arv.reindex(columns = columns)
children_on_arv.columns = ['_'.join(col) for col in children_on_arv.columns.values]
children_on_arv.fillna(0, inplace = True)
children_on_arv.reset_index(inplace = True)
children_on_arv

#Dataset children not on arv
hhm_children_not_on_arv = hhm_positive_children[hhm_positive_children.on_arv == '---']

children_not_on_arv = hhm_children_not_on_arv.pivot_table(values = 'patient_code', index = 'Commune',
                                                         columns = ['on_arv','gender','age_range'], aggfunc = len, fill_value = 0)

children_not_on_arv.rename(columns = {'---': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
children_not_on_arv = children_not_on_arv.reindex(columns = columns)
children_not_on_arv.columns = ['_'.join(col) for col in children_not_on_arv.columns.values]
children_not_on_arv.fillna(0, inplace = True)
children_not_on_arv.reset_index(inplace = True)
children_not_on_arv


,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17
0,Delmas,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17
0,Gros-Morne,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0


### No HIV reported (Status Unknown)

In [79]:
#Positive children not on HIV Haiti
hhm_positive_children_not_on_hivhaiti = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'positif') & (hhm_ff_stat.on_HIVHAITI == 'no')]

positive_children_not_on_hivhaiti = hhm_positive_children_not_on_hivhaiti.pivot_table(values = 'patient_code', index = 'Commune',
                                                                                     columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len,
                                                                                     fill_value = 0)
positive_children_not_on_hivhaiti.rename(columns = {'positif': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
positive_children_not_on_hivhaiti = positive_children_not_on_hivhaiti.reindex(columns = columns)
positive_children_not_on_hivhaiti.columns = ['_'.join(col) for col in positive_children_not_on_hivhaiti.columns.values]
positive_children_not_on_hivhaiti.fillna(0, inplace = True)
positive_children_not_on_hivhaiti.reset_index(inplace = True)

#========================================================================================================================
#Children unknown status
status_unknown = hhm_ff_stat[(~hhm_ff_stat.hiv_test_result.isin(['positif', 'negatif']))]

status_unknown.loc[(~status_unknown.hiv_test_result.isin(['positif', 'negatif'])), 'hiv_test_result'] = '---'

children_unknown_status = status_unknown.pivot_table(values = 'patient_code', index = 'Commune', columns = ['hiv_test_result', 'gender','age_range'],
                                                    aggfunc = len, fill_value = 0)
children_unknown_status.rename(columns = {'---': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
children_unknown_status = children_unknown_status.reindex(columns = columns)
children_unknown_status.columns = ['_'.join(col) for col in children_unknown_status.columns.values]
children_unknown_status.fillna(0, inplace = True)
children_unknown_status.reset_index(inplace = True)


#Sum
# children_unknown_status = children_unknown_status.merge(positive_children_not_on_hivhaiti, on = 'Commune', how = 'outer')
# children_unknown_status['No_HIV_reported_Status_Unknown'] = children_unknown_status[['No HIV reported (status Unknown_without positive not on Hiv Haiti)', 'No HIV reported (positive_not_on_hivhaiti)']].sum(axis = 1)
# children_unknown_status.drop(['No HIV reported (status Unknown_without positive not on Hiv Haiti)', 'No HIV reported (positive_not_on_hivhaiti)'], axis = 1, inplace = True)
# children_unknown_status



In [80]:
hhm_positive_children_not_on_hivhaiti.shape[0]
status_unknown.shape[0]


41

296

In [81]:
#SUM
children_status_unknown = pd.concat([positive_children_not_on_hivhaiti,children_unknown_status], ignore_index=True)
children_status_unknown = children_status_unknown.groupby(["Commune"]).sum()
children_status_unknown.reset_index(inplace = True)
children_status_unknown.head()
children_status_unknown.shape


,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse-d'Hainault,0.0,0,0,1,1,0,0,1,1,0
1,Anse-à-Foleur,0.0,0,0,1,0,0,0,0,1,0
2,Anse-à-Galets,0.0,1,0,0,1,0,0,2,2,2
3,Aquin,0.0,0,0,1,0,0,0,0,1,2
4,Arcahaie,0.0,0,0,0,3,0,0,0,0,0


(41, 11)

In [83]:
# children_status_unknown.sum()


In [49]:
# children_status_unknown.to_excel("Verification_status_unk_10112022.xlsx", index = False)


In [84]:
#Concatenation _ FINAL DATA _ OVC_HIVSTAT_CHILDREN

# index = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No_HIV_reported_Status_Unknown']
#------------------------------------------------------------------------------------------------------------------------
def ovc_hivstat_children():
    if len(positive_children) != 0 and len(children_on_arv) != 0 and len(children_not_on_arv) != 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) != 0 and len(children_on_arv) == 0 and len(children_not_on_arv) != 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = positive_children.merge(children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_on_arv','test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) != 0 and len(children_on_arv) != 0 and len(children_not_on_arv) == 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_not_on_arv','test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) == 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = negative_children.merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv','test not indicated']] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    elif len(positive_children) == 0 and len(negative_children) == 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = children_status_unknown
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv', 'negative','test not indicated']] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    else:
        return 'HIV test data are not available right now!!'
    
    
#Calling the function
ovc_hivstat_children()


,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17,test not indicated
0,Anse-d'Hainault,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0
1,Anse-à-Foleur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,Anse-à-Galets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,0
3,Aquin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0
4,Arcahaie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0
5,Bainet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,Cap-Haïtien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
7,Capotille,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,Caracol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,Carice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# DATIM Other kids

In [86]:
#Creating Datim table for the other children
hhm_other['gender'].fillna('Male', inplace = True)
index_other = hhm_other.pivot_table(values = 'patient_code', index = 'Commune', columns = ['gender', 'age_range'], 
                                    aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')

columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18 - 20'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18 - 20')]

index_other = index_other.reindex(columns = columns)
index_other[('Female', '18 - 20')] = 0
index_other[('Male', '18 - 20')] = 0
index_other.reset_index(inplace = True)

index_other


gender                  Commune Female                                      \
age_range                          < 1 1 - 4 5 - 9 10 - 14 15 - 17 18 - 20   
0               Anse-d'Hainault      1     0     3       0       3       0   
1                 Anse-à-Foleur      0     0     0       0       0       0   
2                 Anse-à-Galets      0     0     0       0       0       0   
3                   Anse-à-Veau      0     0     0       0       0       0   
4                         Aquin      0     3     0       2       2       0   
5                      Arcahaie      0     3     2       5       2       0   
6                        Bainet      0     0     0       0       0       0   
7                   Cap-Haïtien      0     0     0       0       0       0   
8                     Capotille      0     0     0       0       0       0   
9                       Caracol      0     0     0       0       0       0   
10                       Carice      0     0     0       0       0       0   
11                    Carrefour      0     4     7       2       2       0   
12                    Cavaillon      3     2     2       2       1       0   
13           Croix-des-Bouquets      1     2     4       5       2       0   
14                       Delmas      1     4    22      30      17       0   
15                   Dessalines      1     0     0       0       0       0   
16                       Ennery      0     1     0       0       0       0   
17                     Gonaïves      3     0     0       0       2       0   
18                     Gressier      0     2     1       0       0       0   
19                   Gros-Morne      2     5     3       2       2       0   
20                   Jean-Rabel      0     0     0       0       0       0   
21                      Jérémie      1     0     0       4       2       0   
22                     Kenscoff      0     0     1       1       0       0   
23                      L'Asile      1     0     0       0       0       0   
24                    La Tortue      2     3     3       8       0       0   
25                    Les Cayes      1     2     0       0       0       0   
26                        Limbé      0     3     5       8       7       0   
27                     Limonade      0     0     0       0       0       0   
28                      Léogâne      0     0     2       2       0       0   
29                    Marmelade      0     1     1       2       3       0   
30                     Maïssade      0     1     0       0       0       0   
31                        Milot      1     3     8       5       2       0   
32                    Miragoâne      1     3    11       5       5       0   
33                Mombin-Crochu      0     0     0       0       0       0   
34                Mont-Organisé      0     2     2       0       0       0   
35                  Ouanaminthe      1     1     1       0       1       0   
36         Petit-Trou-de-Nippes      0     0     0       0       0       0   
37                       Pignon      1     5     9      14       3       0   
38                       Pilate      2     5    10      18       4       0   
39               Plaine-du-Nord      0     1     1       0       0       0   
40                    Plaisance      0     0     0       0       0       0   
41                  Port-Margot      0     1     4       5       1       0   
42               Port-au-Prince      0     1     3       6       4       0   
43                 Port-de-Paix      0     5    11       2       3       0   
44          Saint-Louis-du-Nord      0     0     0       0       0       0   
45                   Saint-Marc      0     0     0       0       1       0   
46                Saint-Raphaël      0     1     1       0       2       0   
47                      Tabarre      1     5    13      14      10       0   
48                Terrier-Rouge      3     2     1       2       0       0   
49                 Tro

In [87]:
#Negative other children
hhm_negative_other_children = hhm_other_stat[hhm_other_stat.hiv_test_result == 'negatif']

negative_other_children = hhm_negative_other_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
negative_other_children.rename(columns = {'negatif': 'negative'}, inplace = True)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative_other_children = negative_other_children.reindex(columns = columns)
negative_other_children.columns = ['_'.join(col) for col in negative_other_children.columns.values]
negative_other_children.reset_index(inplace = True)

negative_other_children.head()


,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse-d'Hainault,0,0,1,0,0,0,0,2,0,0
1,Anse-à-Galets,0,0,0,0,0,0,1,3,0,0
2,Aquin,0,1,0,1,2,0,0,1,0,0
3,Arcahaie,0,3,2,3,0,0,0,2,2,1
4,Cap-Haïtien,0,0,0,0,0,0,0,0,4,0


In [89]:
hhm_negative_other_children.shape[0]
# negative_other_children.sum()


385

In [90]:
#Positive other children
# & (closed == 'FALSE')
hhm_other_stat.rename(columns = {'patient_code2':'full_code_patient_menage'}, inplace = True)
hhm_positive_other_children = hhm_other_stat[(hhm_other_stat.hiv_test_result == 'positif') & (hhm_other_stat.on_HIVHAITI == 'yes')].drop_duplicates('full_code_patient_menage')
hhm_positive_other_children = hhm_positive_other_children.merge(regime[['full_code_patient_menage', 'arv_start_date']], 
                                                    on = 'full_code_patient_menage', how = 'left')
hhm_positive_other_children['on_arv'] = hhm_positive_other_children.arv_start_date.map(on_arv)
positive_other_children = hhm_positive_other_children.pivot_table(values = 'patient_code', index = 'Commune', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
positive_other_children.rename(columns = {'positif': 'positive'}, inplace = True)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive_other_children = positive_other_children.reindex(columns = columns)
positive_other_children.fillna(0, inplace = True)
positive_other_children.columns = ['_'.join(col) for col in positive_other_children.columns.values]
positive_other_children.reset_index(inplace = True)
positive_other_children


,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17
0,Delmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0


In [91]:
hhm_positive_other_children.shape[0]


1

In [92]:
#Dataset children on arv
hhm_other_children_on_arv = hhm_positive_other_children[hhm_positive_other_children.on_arv == 'yes']

other_children_on_arv = hhm_other_children_on_arv.pivot_table(values = 'patient_code', index = 'Commune', 
                                                  columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)
other_children_on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
other_children_on_arv = other_children_on_arv.reindex(columns = columns)
other_children_on_arv.columns = ['_'.join(col) for col in other_children_on_arv.columns.values]
other_children_on_arv.reset_index(inplace = True)

other_children_on_arv

#Dataset children not on arv
hhm_other_children_not_on_arv = hhm_positive_other_children[hhm_positive_other_children.on_arv == 'no']

other_children_not_on_arv = hhm_other_children_not_on_arv.pivot_table(values = 'patient_code', index = 'Commune',
                                                         columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)

other_children_not_on_arv.rename(columns = {'no': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
other_children_not_on_arv = other_children_not_on_arv.reindex(columns = columns)
other_children_not_on_arv.columns = ['_'.join(col) for col in other_children_not_on_arv.columns.values]
other_children_not_on_arv.reset_index(inplace = True)
other_children_not_on_arv


,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17
0,Delmas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [93]:
#Positive other_children not on HIV Haiti
hhm_positive_other_children_not_on_hivhaiti = hhm_other_stat[(hhm_other_stat.hiv_test_result == 'positif') & (hhm_other_stat.on_HIVHAITI == 'no')]

positive_other_children_not_on_hivhaiti = hhm_positive_other_children_not_on_hivhaiti.pivot_table(values = 'patient_code', index = 'Commune',
                                                                                     columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len,
                                                                                     fill_value = 0)
positive_other_children_not_on_hivhaiti.rename(columns = {'positif': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
positive_other_children_not_on_hivhaiti = positive_other_children_not_on_hivhaiti.reindex(columns = columns)
positive_other_children_not_on_hivhaiti.columns = ['_'.join(col) for col in positive_other_children_not_on_hivhaiti.columns.values]
positive_other_children_not_on_hivhaiti.fillna(0, inplace = True)
positive_other_children_not_on_hivhaiti.reset_index(inplace = True)

#other_children unknown status
other_status_unknown = hhm_other[(~hhm_other.hiv_test_result.isin(['positif', 'negatif'])) & (hhm_other.age_in_year <= 17)]

other_status_unknown.loc[(~other_status_unknown.hiv_test_result.isin(['positif', 'negatif'])), 'hiv_test_result'] = '---'
other_children_unknown_status = other_status_unknown.pivot_table(values = 'patient_code', index = 'Commune', columns = ['hiv_test_result', 'gender','age_range'],
                                                    aggfunc = len, fill_value = 0)
other_children_unknown_status.rename(columns = {'---': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
other_children_unknown_status = other_children_unknown_status.reindex(columns = columns)
other_children_unknown_status.columns = ['_'.join(col) for col in other_children_unknown_status.columns.values]
other_children_unknown_status.fillna(0, inplace = True)
other_children_unknown_status.reset_index(inplace = True)

#=========================================================================================================
positive_other_children_not_on_hivhaiti.head()


,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Delmas,0,0,0,0,0,0.0,0,0,0,1
1,Gonaïves,1,0,0,0,0,0.0,0,0,0,0
2,Gros-Morne,1,0,0,0,0,0.0,0,0,0,0
3,La Tortue,1,0,0,1,0,0.0,0,0,0,0
4,Limbé,0,1,0,0,0,0.0,0,0,0,1


In [94]:
hhm_positive_other_children_not_on_hivhaiti.shape[0]
other_status_unknown.shape[0]


15

522

In [95]:
#SUM
other_unknown_status_children = pd.concat([positive_other_children_not_on_hivhaiti, other_children_unknown_status], ignore_index=True)
other_unknown_status_children = other_unknown_status_children.groupby(["Commune"]).sum()
other_unknown_status_children.reset_index(inplace = True)
other_unknown_status_children.head()
other_unknown_status_children.shape


,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse-d'Hainault,1,0,2,0,3,0.0,9,6,0,1
1,Anse-à-Foleur,0,0,0,0,0,0.0,0,1,0,0
2,Anse-à-Veau,0,0,0,0,0,0.0,1,0,0,0
3,Aquin,0,2,0,1,0,0.0,0,2,0,1
4,Arcahaie,0,0,0,2,2,0.0,0,0,3,0


(39, 11)

In [96]:
other_unknown_status_children.sum()


Commune                                          Anse-d'HainaultAnse-à-FoleurAnse-à-VeauAquinAr...
No_HIV_reported_Status_Unknown_Female_< 1                                                       25
No_HIV_reported_Status_Unknown_Female_1 - 4                                                     40
No_HIV_reported_Status_Unknown_Female_5 - 9                                                     70
No_HIV_reported_Status_Unknown_Female_10 - 14                                                   83
No_HIV_reported_Status_Unknown_Female_15 - 17                                                   52
No_HIV_reported_Status_Unknown_Male_< 1                                                       10.0
No_HIV_reported_Status_Unknown_Male_1 - 4                                                       47
No_HIV_reported_Status_Unknown_Male_5 - 9                                                       77
No_HIV_reported_Status_Unknown_Male_10 - 14                                                     99
No_HIV_rep

In [97]:
# index2 = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No HIV reported (Status Unknown)']

#--------------------------------------------------------------------------------------------------------------------
#Concatenation _ OVC_HIVSTAT_OTHER_CHILDREN

def ovc_hivstat_other_children():
    if len(positive_other_children) != 0 and len(other_children_on_arv) != 0 and len(other_children_not_on_arv) != 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_on_arv, on = 'Commune', how = 'outer').merge(other_children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) != 0 and len(other_children_on_arv) == 0 and len(other_children_not_on_arv) != 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['other_children_on_arv','test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) != 0 and len(other_children_on_arv) != 0 and len(other_children_not_on_arv) == 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['other_children_not_on_arv','test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) == 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = negative_other_children.merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children[['positive','on arv', 'not on arv','test not indicated']] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)
        return ovc_hivstat_other_children
    elif len(positive_other_children) == 0 and len(negative_other_children) == 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = other_unknown_status_children
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children[['positive','on arv', 'not on arv', 'negative','test not indicated']] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)
        return ovc_hivstat_other_children
    else:
        return 'HIV test data are not available right now!!'


#Calling the function
ovc_hivstat_other_children()
        

,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17,"(other_children_on_arv, test not indicated)"
0,Anse-d'Hainault,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,3.0,0.0,9.0,6.0,0.0,1.0,0
1,Anse-à-Foleur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,Anse-à-Galets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,Anse-à-Veau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,Aquin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0
5,Arcahaie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0
6,Cap-Haïtien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0
7,Carrefour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0
8,Cavaillon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,0.0,0
9,Croix-des-Bouquets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,3.0,0.0,0


# Caregivers

In [98]:
hh_caregiver = hhm_club[(hhm_club['caregiver_yes_no'] == str(1)) & (hhm_club['age_in_year'] >= 18)]
hh_caregiver.loc[(hh_caregiver.caregiver_yes_no == str(1)), 'caregiver_yes_no'] = 'yes'
hh_caregiver['age_range'] = '18+'
hh_caregiver = hh_caregiver.drop_duplicates('patient_code').reset_index(drop = True)
print(f"Number of caregivers: {hh_caregiver.shape[0]}")


Number of caregivers: 1001


In [99]:
hh_caregiver.to_excel('Caregiver_OEV_FY23Q4_10172023.xlsx', index = False)


In [100]:
caregiver = hh_caregiver.pivot_table(index = 'Commune', values = 'patient_code', columns = ['gender', 'age_range'], 
                                    aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
caregiver.drop('Total', inplace = True)
# caregiver.rename(columns = {'yes': 'Caregiver_18+'}, inplace = True)
caregiver.reset_index(inplace = True)
caregiver


gender,Commune,Female,Male,Total
age_range,,18+,18+,
0,Anse-d'Hainault,9,4,13
1,Anse-à-Foleur,3,0,3
2,Anse-à-Galets,11,7,18
3,Anse-à-Veau,1,0,1
4,Aquin,16,5,21
5,Arcahaie,3,2,5
6,Bainet,3,1,4
7,Cap-Haïtien,8,4,12
8,Capotille,1,1,2


In [101]:
caregiver['Total'].sum()


1001

# Export

In [102]:
import openpyxl

with pd.ExcelWriter('DATIM_OEV_FY23Q4_10172023.xlsx', engine = 'openpyxl') as writer:
    oev.to_excel(writer, sheet_name = 'DATIM_OEV')
    ovc_hivstat.to_excel(writer, sheet_name = 'OVC_HIVSTAT')
    index_ff.to_excel(writer, sheet_name = 'siblings')
    ovc_hivstat_children().to_excel(writer, sheet_name = 'HIV_STAT_Direct')
    index_other.to_excel(writer, sheet_name = 'Index_other')
    ovc_hivstat_other_children().to_excel(writer, sheet_name = 'HIV_STAT_Other')
    caregiver.to_excel(writer, sheet_name = 'Caregivers_18+')


## Closing the connections...

In [103]:
driver.quit()
engine.dispose()


                                                                                                            The end